In [1]:
import os
os.chdir('../')

In [4]:
import numpy as np
import pandas as pd

data=pd.read_csv('Datasets/spotify_data.csv')

Question 1

In [5]:
data.columns

Index(['id', 'performer', 'song', 'genre', 'track_duration', 'track_explicit',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'rating', 'genre_country', 'genre_blues',
       'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance', 'genre_pop',
       'genre_rock', 'genre_hip_hop', 'genre_rnb'],
      dtype='object')

In [6]:
data.drop(labels=['id', 'performer', 'song', 'genre', 'key'],axis=1,inplace=True)

y=data['rating']
X=data.drop(labels='rating',axis=1)

y_binned = pd.qcut(data.rating, q = 20)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1031, stratify=y_binned)

np.median(X_train['liveness'])

0.131

Question 2

In [10]:
from sklearn.linear_model import LinearRegression

genre_model=LinearRegression().fit(X_train[['genre_country',
       'genre_blues', 'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance',
       'genre_pop', 'genre_rock', 'genre_hip_hop', 'genre_rnb']],y_train)

from sklearn.metrics import root_mean_squared_error

y_pred_train=genre_model.predict(X_train[['genre_country',
       'genre_blues', 'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance',
       'genre_pop', 'genre_rock', 'genre_hip_hop', 'genre_rnb']])

root_mean_squared_error(y_train,y_pred_train).round(4)

15.8119

Question 3

In [21]:
from sklearn.tree import DecisionTreeRegressor

genre_tree=DecisionTreeRegressor(max_depth=4).fit(X_train[['genre_country',
       'genre_blues', 'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance',
       'genre_pop', 'genre_rock', 'genre_hip_hop', 'genre_rnb']],y_train)

y_pred_tree=genre_tree.predict(X_train[['genre_country',
       'genre_blues', 'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance',
       'genre_pop', 'genre_rock', 'genre_hip_hop', 'genre_rnb']])

root_mean_squared_error(y_train,y_pred_tree)

15.691917118321479

Question 4

In [16]:
from sklearn.ensemble import VotingRegressor

lin_model = LinearRegression()
tree_model = DecisionTreeRegressor(max_depth=4, random_state=1031)


voting_model = VotingRegressor(
    estimators=[
        ('lin', lin_model),
        ('tree', tree_model)
    ]
)


voting_model.fit(X_train[['genre_country',
       'genre_blues', 'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance',
       'genre_pop', 'genre_rock', 'genre_hip_hop', 'genre_rnb']], y_train)


y_pred_voting = voting_model.predict(X_train[['genre_country',
       'genre_blues', 'genre_jazz', 'genre_rap', 'genre_space', 'genre_dance',
       'genre_pop', 'genre_rock', 'genre_hip_hop', 'genre_rnb']])


root_mean_squared_error(y_train, y_pred_voting).round(4)


15.6708

Question 3 and 4 are wrong

In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# ---------------------------------------
# 1. Select only the 10 genre dummy vars
# ---------------------------------------
genre_cols = [
    'genre_country','genre_blues','genre_jazz','genre_rap','genre_space',
    'genre_dance','genre_pop','genre_rock','genre_hip_hop','genre_rnb'
]

X_train_genre = X_train[genre_cols]

# ---------------------------------------
# 2. Build tree model with max_depth = 4
# ---------------------------------------
genre_tree = DecisionTreeRegressor(max_depth=4, random_state=1031)
genre_tree.fit(X_train_genre, y_train)

# ---------------------------------------
# 3. Predict on TRAIN sample
# ---------------------------------------
y_pred_tree = genre_tree.predict(X_train_genre)

# ---------------------------------------
# 4. Compute RMSE
# ---------------------------------------
rmse_train = mean_squared_error(y_train, y_pred_tree, squared=False)
rmse_train.round(4)


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


15.6919

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# ---------------------------------------
# 1. Tree model uses only genre variables
# ---------------------------------------
genre_cols = [
    'genre_country','genre_blues','genre_jazz','genre_rap','genre_space',
    'genre_dance','genre_pop','genre_rock','genre_hip_hop','genre_rnb'
]

X_train_genre = X_train[genre_cols]

tree_model = DecisionTreeRegressor(max_depth=4, random_state=1031)
tree_model.fit(X_train_genre, y_train)

# ---------------------------------------
# 2. Linear Regression model (uses all X_train)
# ---------------------------------------
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

# ---------------------------------------
# 3. Combine using Voting Regressor
# ---------------------------------------
voting_model = VotingRegressor(
    estimators=[
        ('lin', lin_model),
        ('tree', tree_model)
    ]
)

# IMPORTANT: VotingRegressor expects both models to use the SAME X
voting_model.fit(X_train, y_train)

# ---------------------------------------
# 4. Predict on TRAIN sample
# ---------------------------------------
y_pred_voting = voting_model.predict(X_train)

# ---------------------------------------
# 5. Compute TRAIN RMSE
# ---------------------------------------
rmse_train_voting = root_mean_squared_error(y_train, y_pred_voting)
rmse_train_voting.round(4)


15.2625